In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import pathlib
from tqdm import tqdm

# Dataset Serie A matches
Per la realizzazione del dataset ho trovato in letteratura l'elenco dei dati più importanti per la predizione dei risultati delle partite. 

https://iopscience.iop.org/article/10.1088/1757-899X/226/1/012099/pdf

Per questo dataset ho utilizzato le statistiche della stagione 2021/22

In [31]:
headers = {'User-Agent': 
                'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

page = "https://fbref.com/it/comp/11/2021-2022/Statistiche-di-Serie-A-2021-2022"
pageTree = requests.get(page)
soup = BeautifulSoup(pageTree.text, features="lxml")

In [32]:
table = soup.select('table.stats_table')[0]
a_hrefs = table.find_all('a')

In [33]:
home_main_factors = ["team", "goals", "total_shots", "shots_on_target", "goals_on_penalty", "total_penalties", "free_kick", "corners", "yellow_cards", "red_cards", "fouls", "completed_passings",  "total_passings", "percentage_possession"]

away_main_factors = ["team", "goals", "total_shots", "shots_on_target", "goals_on_penalty", "total_penalties", "free_kick", "corners", "yellow_cards", "red_cards", "fouls", "completed_passings",  "total_passings", "percentage_possession"]

home_main_factors = ["home_"+factor for factor in home_main_factors]
away_main_factors = ["away_"+factor for factor in away_main_factors]

In [34]:
teams = [a_href for a_href in a_hrefs if '/squadre/' in str(a_href)]        
#for team in teams:
teams.sort(key=lambda x: x.contents[0]) #ordinamento per titolo

In [17]:
#questo dizionario contiene come chiave i link ai bottoni che permettono di portare ai link dei vari dataset della squadra e come valore il nome della parte di html che permette di accedere al dataset
util_hrefs = {
    'all_comps/shooting/': {
        'section': 'Tiri',
        'columns': ['Data', 'Tiri','Tiri.1','Rigori','Rig T']
        },
    'all_comps/passing/': {
        'section': 'Passaggi',
        'columns': ['Data', 'Compl.', 'Tent,']
        },
        
    'all_comps/passing_types': {
        'section': 'Tipologie di passaggi',
        'columns': ['Data', 'Angoli']
        },
    'all_comps/possession': {
        'section': 'Possesso palla',
        'columns': ['Data', 'Poss.']
        },
    'all_comps/misc': {
        'section': 'Statistiche varie',
        'columns': ['Data', 'Falli', 'Amm.', 'Esp.']
        },
}

In [35]:
datasets = pd.DataFrame()
home_games = pd.DataFrame()
away_games = pd.DataFrame()

In [36]:
from queue import Empty

for team in tqdm(teams[:2]):
    team_name = team.contents[0]
    link = f"https://fbref.com{team.get('href')}"
    data = requests.get(link)
    games = pd.read_html(data.text, match="Punteggi e partite")[0]

    matches = games[games['Competizione'] == 'Serie A']
    matches = matches[['Data', 'Stadio', 'Risultato', 'Rf','Avversario','Rs']]
    #match[home_main_factors[0]] = team_name
    matches.insert(0, home_main_factors[0], team_name)
    matches = matches.rename(columns={'Rf': home_main_factors[1], 'Avversario': away_main_factors[0], 'Rs': away_main_factors[1]})
    
    #ottengo il dataset della singola squadra con tutti i dati che servono
    soup = BeautifulSoup(data.text, features="lxml")
    links = soup.find_all('a')

    href_links = [l.get("href") for l in links]

    old_dataset = matches
    for href_key, section_value in util_hrefs.items():
        div_links = [l for l in href_links if l and href_key in l]
        #prendo il link e ottengo l'html
        html = requests.get(f"https://fbref.com{div_links[0]}")
        
        section, columns = section_value.items()
        section, columns = section[1], columns[1]
        #ottengo il dataset della pima sezione indicata nel match
        dataset_section = pd.read_html(html.text, match=section)[0]
        #elimino l'intestazione "Di: NomeSquadra"
        dataset_section.columns = dataset_section.columns.droplevel()
        #filtro il dataset per la competizione
        dataset_section = dataset_section[dataset_section['Competizione'] == 'Serie A']
        #filtro il dataset per le colonne selezionate
        dataset_section = dataset_section[[col for col in columns]]
        #print(dataset_section.columns)
        if section == 'Passaggi':
            column_names = dataset_section.columns.values
            column_names[1] = 'Compl'
            column_names[5] = 'Tent'
            dataset_section = dataset_section[['Data','Compl','Tent']]

        old_dataset = pd.merge(old_dataset, dataset_section, on = ["Data"])
    
    datasets = datasets.append(old_dataset)

100%|██████████| 20/20 [00:56<00:00,  2.84s/it]


Rinomino tutte le colonne considerando come se tutte le squadre giocassero in casa

In [42]:
home_games = datasets[datasets['Stadio']=='Casa']
away_games = datasets[datasets['Stadio']=='Ospiti']

In [43]:
home_dataset = home_games.rename(columns={'Tiri': home_main_factors[2], 'Tiri.1': home_main_factors[3], 'Rigori': home_main_factors[4], 'Rig T': home_main_factors[5], 'Compl': home_main_factors[11], 'Tent': home_main_factors[12], 'Angoli': home_main_factors[7], 'Poss.': home_main_factors[13], 'Falli': home_main_factors[10], 'Amm.': home_main_factors[8], 'Esp.': home_main_factors[9]})

away_dataset = away_games.rename(columns={'home_team': away_main_factors[0], 'away_team': home_main_factors[0], 'home_goals': away_main_factors[1], 'away_goals': home_main_factors[1], 'Tiri': away_main_factors[2], 'Tiri.1': away_main_factors[3], 'Rigori': away_main_factors[4], 'Rig T': away_main_factors[5], 'Compl': away_main_factors[11], 'Tent': away_main_factors[12], 'Angoli': away_main_factors[7], 'Poss.': away_main_factors[13], 'Falli': away_main_factors[10], 'Amm.': away_main_factors[8], 'Esp.': away_main_factors[9]})


In [44]:
home_dataset.to_csv("homes.csv")
away_dataset.to_csv("aways.csv")